### Using a TRILEGAL file for a given healpix, produce maps with Mr - [Fe/H] priors 

Version with a subsample of the hipscat TRILEGAL catalog processed with LSDB

In [1]:
%matplotlib inline
import pandas as pd
from dask.distributed import Client
pd.set_option('display.max_columns', None)

In [2]:
import os
import shutil
from collections import defaultdict

def delete_files_and_directories(directory):
    # Dictionary to store the count of each file extension
    extension_counts = defaultdict(int)
    file_count = 0
    dir_count = 0
    
    # Walk through the directory
    for root, dirs, files in os.walk(directory, topdown=False):
        # Delete files and collect their extensions
        for file in files:
            file_path = os.path.join(root, file)
            file_extension = os.path.splitext(file)[1]  # Get file extension
            extension_counts[file_extension] += 1
            os.remove(file_path)
            file_count += 1
        
        # Delete directories
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            shutil.rmtree(dir_path)
            dir_count += 1
    
    # Print the results
    print(f'Total files deleted: {file_count}')
    print(f'Total directories deleted: {dir_count}')
    print('File extensions and their counts:')
    for ext, count in extension_counts.items():
        print(f'{ext}: {count}')
    
    return file_count, dir_count, dict(extension_counts)

In [3]:
client = Client(n_workers=192, threads_per_worker=1, memory_limit="128GB", dashboard_address=':41987')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:41987/status,
Dashboard: http://127.0.0.1:41987/status,Workers: 192
Total threads: 192,Total memory: 22.35 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:32985,Workers: 192
Dashboard: http://127.0.0.1:41987/status,Total threads: 192
Started: Just now,Total memory: 22.35 TiB
Comm: tcp://127.0.0.1:34749,Total threads: 1
Dashboard: http://127.0.0.1:44073/status,Memory: 119.21 GiB
Nanny: tcp://127.0.0.1:42779,


## LSDB

In [4]:
%%time
import lsdb

trilegal = lsdb.read_hats(
    "/mnt/beegfs/scratch/data/TRILEGAL/TRILEGAL_hats/TRILEGAL_level_5_cluster/",
    # search_filter=lsdb.BoxSearch(ra=(-53, 61), dec=(-2., 2.)),
    search_filter=None,
)

# trilegal.skymap(lambda df, a: np.log10(len(df)+1), order=9)

CPU times: user 2 s, sys: 599 ms, total: 2.6 s
Wall time: 2.23 s


In [5]:
%%time

from photod import priors

def prep(df, pix):
    subdir = f'{rootdir}/{str(pix.order)}'
    rootname = f'{subdir}/{str(pix.pixel)}'
    
    if not os.path.exists(subdir):
        os.makedirs(subdir)
        print(f'Directory created: {subdir}')
    else:
        print(f"Directory already exists: {subdir}")


    priors.dumpPriorMaps_testing(df, rootname, pix, verbose=False)
    return pd.DataFrame(data=[(pix, len(df))], columns=['HealPix', 'len'])

rootdir = '/mnt/beegfs/scratch/kmalanch/photod/priors/TRILEGAL'  # TODO: clean this up!

# Perform the deletion and gather statistics
delete_files_and_directories(rootdir)
trilegal.map_partitions(prep, include_pixel=True).compute()

Total files deleted: 0
Total directories deleted: 0
File extensions and their counts:
Directory created: /mnt/beegfs/scratch/kmalanch/photod/priors/TRILEGAL/0
sample <class 'nested_pandas.nestedframe.core.NestedFrame'>
Healpix:  Order: 0, Pixel: 0 
---------------------------------------
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR: no data to make map (see dumpPriorMaps)
ERROR:

,HealPix,len
0,"Order: 5, Pixel: 0",72298
0,"Order: 5, Pixel: 1",74223
0,"Order: 5, Pixel: 2",72682
0,"Order: 5, Pixel: 3",73456
0,"Order: 5, Pixel: 4",75190
...,...,...
0,"Order: 5, Pixel: 12283",374916
0,"Order: 5, Pixel: 12284",311610
0,"Order: 5, Pixel: 12285",293858
0,"Order: 5, Pixel: 12286",343927


Wall time: 56m 8s

In [6]:
client.close()

Directory already exists: /mnt/beegfs/scratch/kmalanch/photod/priors/TRILEGAL/5
sample <class 'nested_pandas.nestedframe.core.NestedFrame'>
Healpix:  Order: 5, Pixel: 4232 
---------------------------------------
Directory already exists: /mnt/beegfs/scratch/kmalanch/photod/priors/TRILEGAL/5
sample <class 'nested_pandas.nestedframe.core.NestedFrame'>
Healpix:  Order: 5, Pixel: 12144 
---------------------------------------
Directory already exists: /mnt/beegfs/scratch/kmalanch/photod/priors/TRILEGAL/5
sample <class 'nested_pandas.nestedframe.core.NestedFrame'>
Healpix:  Order: 5, Pixel: 11947 
---------------------------------------
Directory already exists: /mnt/beegfs/scratch/kmalanch/photod/priors/TRILEGAL/5
sample <class 'nested_pandas.nestedframe.core.NestedFrame'>
Healpix:  Order: 5, Pixel: 4164 
---------------------------------------
Directory already exists: /mnt/beegfs/scratch/kmalanch/photod/priors/TRILEGAL/5
sample <class 'nested_pandas.nestedframe.core.NestedFrame'>
Healpi

2024-12-14 01:55:09,693 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-12-14 01:55:09,698 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-12-14 01:55:09,700 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-12-14 01:55:09,702 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-12-14 01:55:09,703 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-12-14 01:55:09,703 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-12-14 01:55:09,704 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-12-14 01:55:09,705 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-12-14 01:55:09,705 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-12-14

TimeoutError: 